In [0]:
%sql
create schema if not exists bike_data.silver

In [0]:
df=spark.table('bike_data.bronze.cust_az12_erp')

In [0]:
df.display()

### Checking Dupliactes

In [0]:
from pyspark.sql.functions import *

In [0]:
df.groupBy("CID") \
  .count() \
  .filter("count > 1") \
  .display()

### Handling Null Values 

In [0]:
from pyspark.sql.functions import *

df.select(
    *[
        sum(col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ]
).display()

In [0]:
df.count()

In [0]:
df.describe()

In [0]:
df.display()

In [0]:
df.schema

#### Changing Column names

In [0]:
df.columns

In [0]:
rename_map = {
    "CID": "customer_key",
    "BDATE": "birthdate",
    "GEN": "gender"
}


In [0]:
for old, new in rename_map.items():
    df = df.withColumnRenamed(old, new)

df.columns


In [0]:
from pyspark.sql.functions import col

df=df.withColumn(
    "customer_id",
    col("customer_key").substr(9, 11)
)

In [0]:
df.display()

In [0]:
# 10. Write Silver
df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("bike_data.silver.dim_customer_genral_details")